In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn import svm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

## Lendo os dados que foram obtidos por script python através de scrapping

In [37]:
df = pd.read_csv('./data/others/querobolsa.csv')

# Aqui é um exemplo pequeno de como funciona o nosso DataFrame (Tabela)

## Unammed: 0 é a coluna que atribui um id automatico e auto incremental para cada linha
## author: representa o nome de quem escreveu a avaliação no site
## text: uma concatenação do texto de pros e contras
## rating: Classificação de impressão positiva ou negativa do usuário em relação a universidade, baseada na nota da plataforma.

### Se a nota na plataforma for maior que 3, então é Positivo. Se for menor ou igual a 2, então Negativo. Se estiver entre esses valores, então a nota é Neutro.

In [38]:
df.head()

,Unnamed: 0,author,text,rating
0,0,Arthur Menisck,A Universidade Paulista - UNIP tem melhorado b...,Neutro
1,1,Bruna Bianchini,Faculdade extremamente fácil de localizar e su...,Positivo
2,2,Bruna Rodrigues,"Professores ótimos, excelente nota no MEC, boa...",Positivo
3,3,Márcia Dias,O ensino é o ponto mais positivo da universida...,Positivo
4,4,Eugênio Castro,Gostei do local tem boa estrutura acesso fácil...,Positivo


In [39]:
df[df.rating=='Neutro'].count()

Unnamed: 0    697
author        696
text          697
rating        697
dtype: int64

In [40]:
df[df.rating=='Positivo'].count()

Unnamed: 0    3200
author        3199
text          3200
rating        3200
dtype: int64

In [41]:
df[df.rating=='Negativo'].count()

Unnamed: 0    103
author        103
text          103
rating        103
dtype: int64

In [42]:
texto = df['text'].values
classificacao = df['rating'].values

# Bag of Words (BoW) - Processo de Vetorização

## 1. Tokenização:

O primeiro passo é dividir o texto em unidades menores chamadas "tokens". Em BoW, os tokens geralmente são palavras, mas podem ser n-gramas ou até mesmo caracteres, dependendo da granularidade desejada.

**Exemplo:**

Texto: "A análise de sentimentos é uma parte interessante do processamento de linguagem natural."

Tokens: ["A", "análise", "de", "sentimentos", "é", "uma", "parte", "interessante", "do", "processamento", "de", "linguagem", "natural"]


## 2. Construção do Vocabulário:

Em seguida, cria-se um vocabulário único a partir de todos os tokens encontrados no corpus (conjunto de documentos). Cada palavra única no vocabulário recebe um índice único.

**Exemplo:**

Vocabulário: {"A": 0, "análise": 1, "de": 2, "sentimentos": 3, "é": 4, "uma": 5, "parte": 6, "interessante": 7, "do": 8, "processamento": 9, "linguagem": 10, "natural":11}

## 3. Contagem de Ocorrências:

Para cada documento, conta-se quantas vezes cada palavra do vocabulário aparece no documento. Essa contagem é armazenada em um vetor

**Exemplo:**

Vetor do Documento: [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Este vetor representa a contagem de ocorrências de cada palavra no documento em relação ao vocabulário.

## 4. Vetorização Final:

O conjunto de vetores resultante, um para cada documento, forma a representação final do BoW para o corpus.

**Exemplo (Dois Documentos):**

Documento 1: [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Documento 2: [0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 1]

Cada posição no vetor representa a contagem de ocorrências de uma palavra específica no documento.

## Considerações Finais:

- BoW perde a ordem das palavras e considera apenas a presença ou ausência delas.
- Pode resultar em vetores esparsos, especialmente em grandes vocabulários.
- Pode ser usadgoritmos de aprendizado.
.
tos):**
lo:**
:**
]


In [43]:
vectorizer = CountVectorizer(analyzer="word")
freq_texto = vectorizer.fit_transform(texto)
modelo = MultinomialNB()
modelo.fit(freq_texto, classificacao)

MultinomialNB()

In [44]:
freq_texto.shape

(4000, 9947)

In [45]:
freq_texto.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [46]:
testes = ['Essa faculdade está um lixo', 'Essa faculdade é boa', 'Essa faculdade não é boa', 'Não sei se amo ou odeio essa faculdade', 'Amo a UNIP']

In [47]:
freq_testes = vectorizer.transform(testes)

In [48]:
for t, c in zip(testes, modelo.predict(freq_testes)):
    print(f'{t}, {c}')

Essa faculdade está um lixo, Positivo
Essa faculdade é boa, Positivo
Essa faculdade não é boa, Positivo
Não sei se amo ou odeio essa faculdade, Positivo
Amo a UNIP, Positivo


In [49]:
print(modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

['Negativo' 'Neutro' 'Positivo']


array([[0.01, 0.13, 0.86],
       [0.01, 0.11, 0.88],
       [0.  , 0.11, 0.89],
       [0.01, 0.03, 0.97],
       [0.01, 0.02, 0.97]])

In [50]:
def add_neg(texto):
    neg = ['não', 'not', 'Não']
    neg_detected = False
    res = []
    words = texto.split()
    for p in words:
        p = p.lower()
        if neg_detected == True:
            p = p + '_NEG'
        if p in neg:
            neg_detected = True
        res.append(p)
    return(" ".join(res))

In [51]:
add_neg('Eu gosto da faculade e gostaria de continuar estudando aqui')

'eu gosto da faculade e gostaria de continuar estudando aqui'

In [52]:
add_neg('Eu não gosto da faculdade')

'eu não gosto_NEG da_NEG faculdade_NEG'

In [53]:
simple_pipeline = Pipeline([('counts', CountVectorizer()), ('classifier', MultinomialNB())])

In [54]:
neg_pipeline = Pipeline([('counts', CountVectorizer(tokenizer=lambda text: add_neg(text))), ('classifier', MultinomialNB())])

In [55]:
simple_pipeline.fit(texto, classificacao)

Pipeline(steps=[('counts', CountVectorizer()), ('classifier', MultinomialNB())])

In [56]:
simple_pipeline.steps

[('counts', CountVectorizer()), ('classifier', MultinomialNB())]

In [57]:
neg_pipeline.fit(texto, classificacao)

Pipeline(steps=[('counts',
                 CountVectorizer(tokenizer=<function <lambda> at 0x000002E67C86B490>)),
                ('classifier', MultinomialNB())])

In [58]:
neg_pipeline.steps

[('counts',
  CountVectorizer(tokenizer=<function <lambda> at 0x000002E67C86B490>)),
 ('classifier', MultinomialNB())]

In [59]:
simple_pipeline = Pipeline([('counts', CountVectorizer()), ('classifier', svm.SVC(kernel='linear'))])

In [60]:
neg_pipeline = Pipeline([('counts', CountVectorizer(tokenizer=lambda text: add_neg(text))), ('classifier', svm.SVC(kernel='linear'))])

In [61]:
simple_res = cross_val_predict(simple_pipeline, texto, classificacao, cv=10)

In [62]:
metrics.accuracy_score(classificacao, simple_res)

0.7295

In [63]:
neg_res = cross_val_predict(neg_pipeline, texto, classificacao, cv=10)

In [64]:
metrics.accuracy_score(classificacao, neg_res)

0.7995

In [65]:
simple_svm_pipeline = Pipeline([('counts', CountVectorizer()), ('classifier', svm.SVC(kernel='linear'))])

In [66]:
neg_svm_pipeline = Pipeline([('counts', CountVectorizer(tokenizer=lambda text: add_neg(text))), ('classifier', svm.SVC(kernel='linear'))])

In [67]:
sentimento = ['Positivo', 'Negativo', 'Neutro']
print(metrics.classification_report(classificacao, simple_res))

              precision    recall  f1-score   support

    Negativo       0.22      0.19      0.21       103
      Neutro       0.28      0.27      0.27       697
    Positivo       0.84      0.85      0.84      3200

    accuracy                           0.73      4000
   macro avg       0.44      0.44      0.44      4000
weighted avg       0.72      0.73      0.73      4000



In [68]:
print(metrics.classification_report(classificacao, neg_res))

              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00       103
      Neutro       0.25      0.00      0.00       697
    Positivo       0.80      1.00      0.89      3200

    accuracy                           0.80      4000
   macro avg       0.35      0.33      0.30      4000
weighted avg       0.68      0.80      0.71      4000



In [69]:
print (pd.crosstab(classificacao, neg_res, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Neutro  Positivo   All
Real                            
Negativo       0       103   103
Neutro         1       696   697
Positivo       3      3197  3200
All            4      3996  4000


In [70]:
print('oi')

oi
